In [59]:
import numpy as np
import pandas as pd
path = u"./dataset/國際權威期刊：「吹哨人」揭輝瑞BNT疫苗震驚消息！臨床試驗存在3大問題！打這疫苗後，體內部分免疫力下降？（2021.11.11）_ 健康1+1 · 直播.srt"
x = pd.read_csv(path, sep='\n')
y = np.array(x)

z = []
z.append(np.array(['1']))
for i in y:
    z.append(i)

for i in z:
    print(i)

s = ""
idx = 0
subPack = []
pack = []
for i in z:
    subPack.append(i[0])
    idx += 1
    if(idx % 3 == 0):
        pack.append(subPack)
        subPack = []
        idx = 0

start = []
end = []
for i in range(len(pack)):
    s = pack[i][1]
    s = s.replace(" --> ", "")
    s = s[0:8], s[12:20]
    a = list(s)
    pack[i][1] = a


['1']
['00:00:02,048 --> 00:00:07,936']
['新竹芬蘭距是新盛健康之道大家好歡迎來到健康一加一房一身心靈我是99']
['2']
['00:00:08,192 --> 00:00:11,776']
['今天是美東時間2021年11月11號']
['3']
['00:00:12,032 --> 00:00:18,176']
['今天是美國的老兵節退伍軍人節和兔今年紀念日那戒指機會了健康一加一']
['4']
['00:00:18,432 --> 00:00:24,576']
['放射師精靈全體同仁相退伍軍人以及參加過濾次戰爭的退伍軍人表']
['5']
['00:00:24,832 --> 00:00:25,344']
['打記憶']
['6']
['00:00:25,600 --> 00:00:27,392']
['感謝你們為國家的付出']
['7']
['00:00:27,648 --> 00:00:33,792']
['好啦回到以前的消息美國cdc本週將歐洲兩個國家添加到4級名單']
['8']
['00:00:34,048 --> 00:00:37,376']
['南非常高風險的旅遊目的名單中']
['9']
['00:00:37,632 --> 00:00:41,728']
['截至11月8號cdc的事情名單上有80個國家']
['10']
['00:00:41,984 --> 00:00:46,592']
['其中有包括英國希臘瑞士奧地利土耳其和馬爾代夫']
['11']
['00:00:47,104 --> 00:00:53,248']
['那那麼的那一秒的孔發引發心臟病德國建議呢30歲以下的人不要學']
['12']
['00:00:53,504 --> 00:00:54,272']
['油墨的那一秒']
['13']
['00:00:54,784 --> 00:00:58,880']
['在美國洛杉磯買菜還有用餐要開疫苗施打證明']
['14']
['00:00:59,136 --> 00:01:03,488']
['商家如果是違規的話那最高要罰130000美元']
['15']
['00:01:03,744 --> 00:01:09,888']
['近期國際權威學術期刊英國醫學期刊採訪了一位協助輝瑞作三']
[

In [48]:
pack

[['1', ['00:00:02', '00:00:07'], '新竹芬蘭距是新盛健康之道大家好歡迎來到健康一加一房一身心靈我是99'],
 ['2', ['00:00:08', '00:00:11'], '今天是美東時間2021年11月11號'],
 ['3', ['00:00:12', '00:00:18'], '今天是美國的老兵節退伍軍人節和兔今年紀念日那戒指機會了健康一加一'],
 ['4', ['00:00:18', '00:00:24'], '放射師精靈全體同仁相退伍軍人以及參加過濾次戰爭的退伍軍人表'],
 ['5', ['00:00:24', '00:00:25'], '打記憶'],
 ['6', ['00:00:25', '00:00:27'], '感謝你們為國家的付出'],
 ['7', ['00:00:27', '00:00:33'], '好啦回到以前的消息美國cdc本週將歐洲兩個國家添加到4級名單'],
 ['8', ['00:00:34', '00:00:37'], '南非常高風險的旅遊目的名單中'],
 ['9', ['00:00:37', '00:00:41'], '截至11月8號cdc的事情名單上有80個國家'],
 ['10', ['00:00:41', '00:00:46'], '其中有包括英國希臘瑞士奧地利土耳其和馬爾代夫'],
 ['11', ['00:00:47', '00:00:53'], '那那麼的那一秒的孔發引發心臟病德國建議呢30歲以下的人不要學'],
 ['12', ['00:00:53', '00:00:54'], '油墨的那一秒'],
 ['13', ['00:00:54', '00:00:58'], '在美國洛杉磯買菜還有用餐要開疫苗施打證明'],
 ['14', ['00:00:59', '00:01:03'], '商家如果是違規的話那最高要罰130000美元'],
 ['15', ['00:01:03', '00:01:09'], '近期國際權威學術期刊英國醫學期刊採訪了一位協助輝瑞作三'],
 ['16', ['00:01:10', '00:01:15'], '三期臨床試驗公司的主任他指出輝瑞疫苗在臨床試驗中'],
 ['17', ['00:01:15', '00:01:19'], '存在偽造數據這個

In [4]:
import numpy as np
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.snippets import to_array

config_path = "./model/bert_base/chinese_L-12_H-768_A-12/bert_config.json"
checkpoint_path = './model/bert_base/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = './model/bert_base/chinese_L-12_H-768_A-12/vocab.txt'

In [5]:
model = build_transformer_model(
    config_path=config_path, checkpoint_path=checkpoint_path, with_nsp=True
)


In [6]:
tokenizer = Tokenizer(dict_path, do_lower_case=True)


In [26]:
token=[]
seg=[]
token_ids=""
segment_ids=""
result=[]
for i in range(len(pack)-1):
    token_ids, segment_ids = tokenizer.encode(
        pack[i][2],pack[i+1][2])
    token_ids, segment_ids = to_array([token_ids], [segment_ids])

    probas = model.predict([token_ids, segment_ids])[0]
    result.append(probas)

In [161]:
token_ids

(48,)

In [8]:
token_ids=np.array([[token_ids]])
segment_ids=np.array([[segment_ids]])

In [9]:
token_ids.shape,segment_ids.shape

((1, 1, 24), (1, 1, 24))

In [16]:
probas = model.predict([token_ids, segment_ids])[0]

In [28]:
for i in result:
    print(i[0])

0.84790903
0.9998784
0.9999937
0.95410484
0.06585596
0.9207008
0.9988217
0.9995016
0.99997914
0.92204463
0.9999291
0.08154179
0.99985254
0.8185067
0.9999958
0.99803716
0.99798477
0.92734426
0.9266228
0.05250886
0.0028534418
0.7308202
0.21318735
0.021717224
0.057725694
0.990651
0.87842727
0.02641301
0.8811376
0.9989974
0.99886847
0.9962184
0.951958
0.893852
0.6330961
0.026906477
0.055822477
0.9946681
0.9976004
0.9230136
0.014847688
0.20326741
0.044666383
0.018806852
0.99949527
0.9963637
0.9409513
0.9968761
0.0031549525
0.8504798
0.9991698
0.9995091
0.10766992
0.82068825
0.8145785
0.9996001
0.97018325
0.99458647
0.99933225
0.8677398
0.9522263
0.9869682
0.9998814
0.99999356
0.99281436
0.9300587
0.96746683
0.9795322
0.9977277
0.8121666
0.96247184
0.98933816
0.9960718
0.009469367
0.43209106
0.9966503
0.9586966
0.9181202
0.0055017006
0.10886301
0.6576947
0.9999008
0.97862536
0.98221165
0.1950288
0.9991041
0.68816733
0.81863034
0.95664287
0.950464
0.99495536
0.91606283
0.9826032
0.9999634
0.9

In [29]:
token_ids, segment_ids = tokenizer.encode(
    pack[0][2],
    pack[1][2])
token_ids, segment_ids = to_array([token_ids], [segment_ids])
token_ids.shape
probas = model.predict([token_ids, segment_ids])[0]
probas

array([0.84790903, 0.15209104], dtype=float32)

In [22]:
pack[0][2],pack[1][2]

('新竹芬蘭距是新盛健康之道大家好歡迎來到健康一加一房一身心靈我是99', '今天是美東時間2021年11月11號')

In [25]:
token_ids, segment_ids = to_array([token_ids], [segment_ids])

probas = model.predict([token_ids, segment_ids])[0]
probas

array([0.9804324 , 0.01956764], dtype=float32)

In [60]:
for i in range(len(pack)-1):
    pack[i].append(result[i][0])


In [40]:
len(result)

1175

In [68]:
lessThanHalf=[]
lessThanHalf.append('00:00:00')
for i in range(len(pack)-1):
    if(pack[i][3]<0.5):
        print(i)
        lessThanHalf.append(pack[i][1][1])

4
11
19
20
22
23
24
27
35
36
40
41
42
43
48
52
73
74
78
79
84
100
112
113
114
117
118
125
133
134
135
137
138
143
144
146
147
149
153
154
164
168
169
171
172
178
197
200
215
223
224
225
243
244
245
253
269
270
271
276
282
293
294
300
304
310
313
328
330
331
332
336
338
340
343
344
350
352
354
355
363
367
373
378
379
380
386
392
398
400
401
407
411
420
421
424
431
435
442
444
450
451
466
467
481
485
495
506
511
521
522
533
534
536
537
548
549
553
554
568
569
574
577
582
588
599
605
607
616
620
635
642
647
650
652
656
657
660
663
667
686
687
688
691
701
708
718
720
721
727
728
733
734
735
744
746
748
749
750
753
764
781
786
787
788
789
801
802
803
806
807
812
814
815
821
822
826
832
850
851
857
858
861
871
875
876
898
901
904
907
910
914
925
926
942
952
953
967
983
984
995
1001
1004
1010
1011
1033
1035
1040
1041
1044
1045
1049
1050
1051
1054
1055
1064
1067
1069
1070
1074
1078
1080
1081
1085
1093
1101
1103
1105
1107
1108
1109
1111
1125
1127
1128
1129
1130
1131
1132
1134
1137
1138
1140
114

In [53]:
pack[0][3]

0.84790903

In [55]:
pack[0][1][1]

'00:00:07'

In [70]:
lessThanHalf[1]

'00:00:25'

In [65]:
pack[0]

['1',
 ['00:00:02', '00:00:07'],
 '新竹芬蘭距是新盛健康之道大家好歡迎來到健康一加一房一身心靈我是99',
 0.84790903]

In [66]:
pack[0][1][1]

'00:00:07'